In [1]:
!ls /kaggle/input/tabnet/pytorch-tabnet

MarkupSafe-2.1.5-cp311-cp311-win_amd64.whl
colorama-0.4.6-py2.py3-none-any.whl
filelock-3.14.0-py3-none-any.whl
fsspec-2024.5.0-py3-none-any.whl
intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl
jinja2-3.1.4-py3-none-any.whl
joblib-1.4.2-py3-none-any.whl
mkl-2021.4.0-py2.py3-none-win_amd64.whl
mpmath-1.3.0-py3-none-any.whl
networkx-3.3-py3-none-any.whl
numpy-1.26.4-cp311-cp311-win_amd64.whl
pytorch_tabnet-4.1.0-py3-none-any.whl
scikit_learn-1.5.0-cp311-cp311-win_amd64.whl
scipy-1.13.1-cp311-cp311-win_amd64.whl
sympy-1.12-py3-none-any.whl
tbb-2021.12.0-py3-none-win_amd64.whl
threadpoolctl-3.5.0-py3-none-any.whl
torch-2.3.0-cp311-cp311-win_amd64.whl
tqdm-4.66.4-py3-none-any.whl
typing_extensions-4.12.0-py3-none-any.whl


In [2]:
!pip install pytorch-tabnet --no-index --find-links=file:///kaggle/input/tabnet/pytorch-tabnet

Looking in links: file:///kaggle/input/tabnet/pytorch-tabnet
Processing /kaggle/input/tabnet/pytorch-tabnet/pytorch_tabnet-4.1.0-py3-none-any.whl


In [3]:
import os, glob
import gc
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from typing import Literal

from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.metrics import roc_auc_score, auc

import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
import catboost as cat
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from xgboost import XGBClassifier

import torch
import torch.nn as nn
from torch.utils import data
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer

import pickle
import random

from datetime import datetime


In [4]:
import importlib.util
import sys

def load_module_from_path(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

# loaded_module = load_module_from_path('model', '/kaggle/input/ensemble/other/model_fn/1/model.py')
# from model import Transformer, KAN, TransformerKAN

In [5]:
if os.path.exists('/kaggle/input'):
    PATH_DATASET = Path("/kaggle/input/home-credit-credit-risk-model-stability")
else:
    PATH_DATASET = Path("/kaggle/data/")
PATH_PARQUETS = PATH_DATASET / "parquet_files"
PATH_TRAIN = PATH_PARQUETS / "train"
PATH_TEST = PATH_PARQUETS / "test"

if os.path.exists('/kaggle/input'):
    PATH_PREPARED_DATASET = Path("/kaggle/input/dataset")
else:
    PATH_PREPARED_DATASET = Path("/kaggle/data/preprocess")

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Load dataset

In [6]:
class DatasetConstructor:
    def __init__(self, mode: Literal['train', 'test']):
        self.mode = mode
        self.path = PATH_PARQUETS / mode

    @staticmethod
    def reduce_memory_usage_pl(df):
        """ Reduce memory usage by polars dataframe {df} with name {name} by changing its data types.
            Original pandas version of this function: https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 """
        print(f"Memory usage of dataframe is {round(df.estimated_size('mb'), 2)} MB")
        Numeric_Int_types = [pl.Int8,pl.Int16,pl.Int32,pl.Int64]
        Numeric_Float_types = [pl.Float32,pl.Float64]    
        for col in df.columns:
            try:
                col_type = df[col].dtype
                if col_type == pl.Categorical:
                    continue
                c_min = df[col].min()
                c_max = df[col].max()
                if col_type in Numeric_Int_types:
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df = df.with_columns(df[col].cast(pl.Int8))
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df = df.with_columns(df[col].cast(pl.Int16))
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df = df.with_columns(df[col].cast(pl.Int32))
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df = df.with_columns(df[col].cast(pl.Int64))
                elif col_type in Numeric_Float_types:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df = df.with_columns(df[col].cast(pl.Float32))
                    else:
                        pass
                # elif col_type == pl.Utf8:
                #     df = df.with_columns(df[col].cast(pl.Categorical))
                else:
                    pass
            except:
                pass
        print(f"Memory usage of dataframe became {round(df.estimated_size('mb'), 2)} MB")
        return df

    @staticmethod
    def detect_datetime_cols(df):
        return df.select_dtypes(object).apply(lambda x: pd.to_datetime(x, errors='ignore'), axis=0).select_dtypes(np.datetime64).columns.tolist()
                        
    def _to_pandas(self, df):
        df = df.to_pandas().set_index('case_id')
        df = df.replace([np.inf, -np.inf], np.nan)
        return df

    def merge_static(self, df):
        df_static = (
            pl.concat([pl.scan_parquet(p, low_memory=True) for p in glob.glob(str(self.path / f"{self.mode}_static_0_*"))],how="vertical_relaxed",)
            .with_columns(
                [
                    (pl.col(col).cast(pl.String).str.to_date(strict=False)) 
                    for col in [
                        'datefirstoffer_1144D', 
                        'datelastinstal40dpd_247D',
                        'datelastunpaid_3546854D', 
                        'dtlastpmtallstes_4499206D',
                        'firstclxcampaign_1125D', 
                        'firstdatedue_489D', 
                        'lastactivateddate_801D',
                       'lastapplicationdate_877D', 
                        'lastapprdate_640D', 
                        'lastdelinqdate_224D',
                       'lastrejectdate_50D', 
                        'lastrepayingdate_696D',
                       'maxdpdinstldate_3546855D', 
                        'payvacationpostpone_4187118D',
                       'validfrom_1069D'
                    ]
                ] + [
                    (pl.col(col).cast(pl.String).cast(pl.Categorical))
                    for col in [
                        'bankacctype_710L', 'cardtype_51L', 'credtype_322L',
                       'disbursementtype_67L', 'equalitydataagreement_891L',
                       'equalityempfrom_62L', 'inittransactioncode_186L',
                       'isbidproductrequest_292L', 'isdebitcard_729L',
                       'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M',
                       'lastcancelreason_561M', 'lastrejectcommoditycat_161M',
                       'lastrejectcommodtypec_5251769M', 'lastrejectreason_759M',
                       'lastrejectreasonclient_4145040M', 'lastst_736L', 'opencred_647L',
                       'paytype1st_925L', 'paytype_783L', 'previouscontdistrict_112M',
                       'twobodfilling_608L', 'typesuite_864L'
                    ]
                ]
            )
        )
        return df.join(df_static, how="left", on="case_id")
        
    def merge_static_cb(self, df):
        df_static_cb = (
            pl.scan_parquet(self.path / f"{self.mode}_static_cb_0.parquet", low_memory=True)
            .with_columns(
                [
                    (pl.col(col).cast(pl.String).str.to_date(strict=False)) 
                    for col in [
                        'assignmentdate_238D', 
                        'assignmentdate_4527235D',
                        'assignmentdate_4955616D', 
                        'birthdate_574D', 
                        'dateofbirth_337D',
                        'dateofbirth_342D', 
                        'responsedate_1012D', 
                        'responsedate_4527233D',
                        'responsedate_4917613D'
                    ] 
                ] + [
                    (pl.col(col).cast(pl.String).cast(pl.Categorical))
                    for col in [
                        'description_5085714M', 'education_1103M', 'education_88M',
                       'maritalst_385M', 'maritalst_893M', 'requesttype_4525192L',
                       'riskassesment_302T'
                    ]
                ]
            )
        )
        return df.join(df_static_cb, how="left", on="case_id")
 
    def load(self):
        df = pl.scan_parquet(self.path / f"{self.mode}_base.parquet", low_memory=True).with_columns(
            pl.col("date_decision").str.to_date()
        )
        # Depth=0
        df = self.merge_static(df)
        df = self.merge_static_cb(df)
        
        df =(
            df
            .with_columns(
                pl.col(pl.Float64).cast(pl.Float32),
                pl.col(pl.Int64).cast(pl.Int32),
            )
        )
        df = df.select(~cs.date())
        
        # Drop categorical large-dimension columns
        df = df.drop([
            'lastapprcommoditytypec_5251766M',
             'previouscontdistrict_112M',
             'district_544M',
             'profession_152M',
             'name_4527232M',
             'name_4917606M',
             'employername_160M',
             'classificationofcontr_400M',
             'financialinstitution_382M',
             'contaddr_district_15M',
             'contaddr_zipcode_807M',
             'empladdr_district_926M',
             'empladdr_zipcode_114M',
             'registaddr_district_1083M',
             'registaddr_zipcode_184M',
             'addres_district_368M',
             'addres_zip_823M'])
        df = df.collect()
        df = self.reduce_memory_usage_pl(df)
        df = self._to_pandas(df)
        return df

In [7]:
def _to_pandas(df):
    df = df.to_pandas().set_index('case_id')
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

def reduce_memory_usage_pl(df):
        """ Reduce memory usage by polars dataframe {df} with name {name} by changing its data types.
            Original pandas version of this function: https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 """
        print(f"Memory usage of dataframe is {round(df.estimated_size('mb'), 2)} MB")
        Numeric_Int_types = [pl.Int8,pl.Int16,pl.Int32,pl.Int64]
        Numeric_Float_types = [pl.Float32,pl.Float64]    
        for col in df.columns:
            try:
                col_type = df[col].dtype
                if col_type == pl.Categorical:
                    continue
                c_min = df[col].min()
                c_max = df[col].max()
                if col_type in Numeric_Int_types:
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df = df.with_columns(df[col].cast(pl.Int8))
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df = df.with_columns(df[col].cast(pl.Int16))
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df = df.with_columns(df[col].cast(pl.Int32))
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df = df.with_columns(df[col].cast(pl.Int64))
                elif col_type in Numeric_Float_types:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df = df.with_columns(df[col].cast(pl.Float32))
                    else:
                        pass
                # elif col_type == pl.Utf8:
                #     df = df.with_columns(df[col].cast(pl.Categorical))
                else:
                    pass
            except:
                pass
        print(f"Memory usage of dataframe became {round(df.estimated_size('mb'), 2)} MB")
        return df

In [8]:
test_constructor = DatasetConstructor('test')
df_test = test_constructor.load()
df_test.replace(to_replace=[None], value=np.nan, inplace=True)
df_test
# x_test.info()

Memory usage of dataframe is 0.01 MB
Memory usage of dataframe became 0.01 MB


,MONTH,WEEK_NUM,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastcancelreason_561M,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,riskassesment_302T,riskassesme

In [9]:
train_constructor = DatasetConstructor('train')
df_train = train_constructor.load()
df_train
# data.info()

label = df_train['target']
df_train = df_train.drop(columns=['target'])
df_train

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


Memory usage of dataframe is 1168.76 MB
Memory usage of dataframe became 1160.02 MB


,MONTH,WEEK_NUM,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastcancelreason_561M,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,riskassesment_302T,riskassesme

In [10]:
df_all = pd.concat([df_train, df_test], axis=0)
df_all

/tmp/ipykernel_34/1426741893.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_train, df_test], axis=0)
/tmp/ipykernel_34/1426741893.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_train, df_test], axis=0)
/tmp/ipykernel_34/1426741893.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes.

,MONTH,WEEK_NUM,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastcancelreason_561M,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,riskassesment_302T,riskassesme

In [11]:
filter_columns = ['WEEK_NUM', 'numrejects9m_859L',
 'maxdpdinstlnum_3546846P',
 'pmtaverage_3A',
 'for3years_128L',
 'isbidproductrequest_292L',
 'formonth_535L',
 'numinstpaidlastcontr_4325080L',
 'forquarter_634L',
 'mindbdtollast24m_4525191P',
 'clientscnt_1130L',
 'lastrejectreason_759M',
 'maxdpdlast3m_392P',
 'lastrejectreasonclient_4145040M',
 'disbursementtype_67L',
 'numinstpaid_4499208L',
 'education_1103M',
 'maxdpdlast24m_143P',
 'avgoutstandbalancel6m_4187114A',
 'days90_310L',
 'disbursedcredamount_1113A',
 'numactivecredschannel_414L',
 'thirdquarter_1082L',
 'avgdbdtollast24m_4525197P',
 'currdebt_22A',
 'avgpmtlast12m_4525200A',
 'avgdbddpdlast24m_3658932P',
 'price_1097A',
 'numinstpaidearly5dobd_4499205L',
 'lastotherlnsexpense_631A',
 'lastapprcredamount_781A',
 'cntincpaycont9m_3716944L',
 'deferredmnthsnum_166L',
 'contractssum_5085716L',
 'lastdependentsnum_448L',
 'clientscnt_493L',
 'typesuite_864L',
 'applications30d_658L',
 'equalityempfrom_62L',
 'formonth_206L',
 'firstquarter_103L',
 'forweek_1077L',
 'riskassesment_940T',
 'clientscnt_887L',
 'maxpmtlast3m_4525190A',
 'foryear_818L',
 'numinstpaidlate1d_3546852L',
 'for3years_584L',
 'sellerplacescnt_216L',
 'numinsttopaygr_769L',
 'mindbddpdlast24m_3658935P',
 'lastst_736L',
 'avglnamtstart24m_4525187A',
 'pmtcount_4527229L',
 'clientscnt_533L',
 'applicationscnt_629L',
 'totaldebt_9A',
 'numinstregularpaid_973L',
 'pmtaverage_4527227A',
 'posfpd10lastmonth_333P',
 'avgdbddpdlast3m_4187120P',
 'foryear_850L',
 'pctinstlsallpaidlate1d_3546856L',
 'numactiverelcontr_750L',
 'twobodfilling_608L']

In [12]:
df_all_filtered = df_all[filter_columns]
df_all_filtered


numeric_features = df_all_filtered.select_dtypes(include=np.number).columns
df_all_filtered[numeric_features] = df_all_filtered[numeric_features].apply(
    lambda x: (x-x.mean())/(x.std()))
df_all_filtered[numeric_features] = df_all_filtered[numeric_features].fillna(0)

# cat_features = [col for col in df_all_filtered.columns if df_all_filtered[col].dtype.name == 'category' or df_all_filtered[col].dtype.name == 'object']
# df_all_filtered = pd.get_dummies(df_all_filtered, columns=cat_features, dtype=np.float32)

/tmp/ipykernel_34/3833888713.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_filtered[numeric_features] = df_all_filtered[numeric_features].apply(
/tmp/ipykernel_34/3833888713.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_filtered[numeric_features] = df_all_filtered[numeric_features].fillna(0)


In [13]:
# x_test = x_test[filter_columns]
# data = data[filter_columns]

In [14]:
# x_train, x_valid, y_train, y_valid = train_test_split(data, label, test_size=0.3, shuffle=True, random_state=1293, stratify=label)
from sklearn.preprocessing import LabelEncoder
cv = StratifiedKFold(n_splits=5, shuffle=False)

types = df_all_filtered.dtypes
# df_train = data
weeks = df_train['WEEK_NUM']
df_all_filtered = df_all_filtered.drop(columns=['WEEK_NUM'])

cat_features = [col for col in df_all_filtered.columns if df_all_filtered[col].dtype.name == 'category' or df_all_filtered[col].dtype.name == 'object']

categorical_dims =  {}
for col in df_all_filtered.columns:
    if types[col] == 'object' or types[col] == 'category':
        print(col, df_all_filtered[col].nunique())
        l_enc = LabelEncoder()
#         train[col] = train[col].fillna("VV_likely")
        df_all_filtered[col] = l_enc.fit_transform(df_all_filtered[col].values)
#         categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)

features = [col for col in df_all_filtered.columns] 
cat_idxs = [i for i, f in enumerate(features) if f in cat_features]
cat_dims = [categorical_dims[f] for i, f in enumerate(features) if f in cat_features]

df_train = df_all_filtered.head(len(df_train))
df_test = df_all_filtered.tail(len(df_test))

# df_train[cat_features] = df_train[cat_features].astype(str)
# df_test[cat_features] = df_test[cat_features].astype(str)

isbidproductrequest_292L 2
lastrejectreason_759M 18
lastrejectreasonclient_4145040M 14
disbursementtype_67L 3
education_1103M 5
typesuite_864L 1
equalityempfrom_62L 2
lastst_736L 11
twobodfilling_608L 2


In [15]:
# x_all = pd.concat([x_train, x_valid], axis=0)
# cat_features = [col for col in x_all.columns if x_all[col].dtype == 'category']
# for col in cat_features:
#     x_all[col] = x_all[col].astype('category')
#     x_all[col] = x_all[col].cat.add_categories('Missing').fillna('Missing')
#     x_test[col] = x_test[col].astype('category')
#     x_test[col] = x_test[col].cat.set_categories(x_all[col].cat.categories).fillna('Missing')

# x_train = x_all.iloc[:len(x_train)]
# x_valid = x_all.iloc[len(x_train):]


In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
X_train, X_valid, y_train, y_valid = train_test_split(df_train, label, test_size=0.3) 

tabnet_params = dict(
    n_d = 8,
    n_a = 8,
    n_steps = 5,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = torch.optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = 8787,
    verbose = 2
)

unsupervised_model = TabNetPretrainer(**tabnet_params)

unsupervised_model.fit(
    X_train=X_train.values,
    eval_set=[X_valid.values],
    pretraining_ratio=0.8,
    patience=10
)

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 576.97247| val_0_unsup_loss_numpy: 0.9545400142669678|  0:01:43s
epoch 2  | loss: 1.03936 | val_0_unsup_loss_numpy: 1.1897599697113037|  0:05:11s
epoch 4  | loss: 0.9704  | val_0_unsup_loss_numpy: 0.9103699922561646|  0:08:36s
epoch 6  | loss: 0.99055 | val_0_unsup_loss_numpy: 1.1971800327301025|  0:12:00s
epoch 8  | loss: 1.05973 | val_0_unsup_loss_numpy: 0.9663400053977966|  0:15:23s
epoch 10 | loss: 1.05916 | val_0_unsup_loss_numpy: 0.8922899961471558|  0:18:45s
epoch 12 | loss: 1.11243 | val_0_unsup_loss_numpy: 1.5497699975967407|  0:22:06s
epoch 14 | loss: 1.10183 | val_0_unsup_loss_numpy: 1.0291199684143066|  0:25:26s
epoch 16 | loss: 1.2746  | val_0_unsup_loss_numpy: 1.0593899488449097|  0:28:47s
epoch 18 | loss: 1.04496 | val_0_unsup_loss_numpy: 0.9204800128936768|  0:32:09s
epoch 20 | loss: 1.08782 | val_0_unsup_loss_numpy: 28.975709915161133|  0:35:30s

Early stopping occurred at epoch 20 with best_epoch = 10 and best_val_0_unsup_loss_numpy = 0.89228999614715

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [17]:
from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.2)

fitted_models_cat = []
fitted_models_lgb = []
fitted_models_xgb = []
fitted_models_tabnet = []

cv_scores_cat = []
cv_scores_lgb = []
cv_scores_xgb = []
cv_scores_tabnet = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for idx_train, idx_valid in cv.split(df_train, label, groups=weeks):
    x_train, y_train = df_train.iloc[idx_train], label.iloc[idx_train]
    x_valid, y_valid = df_train.iloc[idx_valid], label.iloc[idx_valid]
    
    
#     x_train_d  = torch.tensor(x_train.values, dtype=torch.float32)
#     x_valid_d  = torch.tensor(x_valid.values, dtype=torch.float32)
#     y_train_d  = torch.tensor(y_train.values, dtype=torch.float32)
#     y_valid_d  = torch.tensor(y_valid.values, dtype=torch.float32)
    
#     # for catboost    
#     train_pool = Pool(x_train, y_train,cat_features=cat_features)
#     val_pool = Pool(x_valid, y_valid,cat_features=cat_features)
    
#     cat_model = CatBoostClassifier(
#         iterations=1200,                 
#         depth=12,                        
#         learning_rate=0.03,               
#         eval_metric='AUC',               
#         random_seed=42,                  
#         bootstrap_type='Bayesian',       
#         bagging_temperature=1,           
#         od_type='Iter',                  
#         od_wait=50,
#         task_type='GPU'
#     )

#     cat_model.fit(
#         x_train, y_train,
#         eval_set=(x_valid, y_valid),
#         cat_features=cat_features,
#         use_best_model=True,
#         verbose=True
#     )
    
#     fitted_models_cat.append(cat_model)
#     y_pred_valid = cat_model.predict_proba(x_valid)[:,1]
#     auc_score = roc_auc_score(y_valid, y_pred_valid)
#     cv_scores_cat.append(auc_score)
    
#     x_train[cat_features] = x_train[cat_features].astype("category")
#     x_valid[cat_features] = x_valid[cat_features].astype("category")
    
#     lgb_params = {
#         "boosting_type": "gbdt",
#         "objective": "binary",
#         "metric": "auc",
#         "max_depth": 10,
#         "num_leaves": 64,
#         "min_data_in_leaf": 10,
#         "learning_rate": 1e-3,
#         "feature_fraction": 0.5,
#         "bagging_fraction": 0.5,
#         "bagging_freq": 5,
#         "n_estimators": 1000,
#         'min_data_in_bin':1,
#         'max_bin': 64,
#         "verbose": -1,
#         "random_state": 42, 
#         'n_jobs': 10
#     }
    
#     lgb_model = lgb.LGBMClassifier(**lgb_params)
#     lgb_model.fit(
#         x_train, y_train,
#         eval_set = [(x_valid, y_valid)],
#         callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)] )
    
#     fitted_models_lgb.append(lgb_model)
#     y_pred_valid = lgb_model.predict_proba(x_valid)[:,1]
#     auc_score = roc_auc_score(y_valid, y_pred_valid)
#     cv_scores_lgb.append(auc_score)
    
#     xgb_model = XGBClassifier(
#         device="cuda",
#         objective='binary:logistic',
#         tree_method="hist",
#         enable_categorical=True,
#         eval_metric='auc',
#         subsample=0.8,
#         colsample_bytree=0.8,
#         min_child_weight=1,
#         max_depth=20,
#         n_estimators=1200,
#         random_state=42,
#     )

#     # Training the model on the training data
#     xgb_model.fit(
#         x_train, y_train,
#         eval_set=[(x_valid, y_valid)],
#         early_stopping_rounds=100,
#         verbose=True,
#     )
#     fitted_models_xgb.append(xgb_model)
#     y_pred_valid = xgb_model.predict_proba(x_valid)[:,1]
#     auc_score = roc_auc_score(y_valid, y_pred_valid)
#     cv_scores_xgb.append(auc_score)
    clf = TabNetClassifier(cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        cat_emb_dim=4,)  #TabNetRegressor()
    
    clf.fit(
        x_train.values, y_train.values,
        eval_set=[(x_valid.values, y_valid.values)],
        eval_metric=["auc"],
#         loss_fn= nn.CrossEntropyLoss(weight=torch.tensor([1., 5.]).to(device)),
        patience=5,
        max_epochs=15,
        weights=1,
        augmentations=aug,
        from_unsupervised=unsupervised_model
    )
    fitted_models_tabnet.append(clf)
    y_pred_valid = clf.predict_proba(x_valid.values)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_tabnet.append(auc_score)
#     preds = clf.predict(X_test)

    
    
    
# print("CV AUC scores: ", cv_scores_cat)
# print("Maximum CV AUC score: ", max(cv_scores_cat))

# print("CV AUC scores: ", cv_scores_lgb)
# print("Maximum CV AUC score: ", max(cv_scores_lgb))

# print("CV AUC scores: ", cv_scores_xgb)
# print("Maximum CV AUC score: ", max(cv_scores_xgb))

print("CV AUC scores: ", cv_scores_tabnet)
print("Maximum CV AUC score: ", max(cv_scores_tabnet))


# with open("fitted_models_cat.pickle", "wb") as f:
#     pickle.dump(fitted_models_cat, f)
# with open("fitted_models_lgb.pickle", "wb") as f:
#     pickle.dump(fitted_models_lgb, f)
# with open("fitted_models_xgb.pickle", "wb") as f:
#     pickle.dump(fitted_models_xgb, f)

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [3, 18, 14, 4, 6, 2, 3, 12, 3] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [4, 4, 4, 4, 4, 4, 4, 4, 4] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [4, 10, 12, 13, 15, 35, 37, 50, 63] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining

epoch 0  | loss: 0.58229 | val_0_auc: 0.68985 |  0:01:27s
epoch 1  | loss: 0.56877 | val_0_auc: 0.69041 |  0:02:57s
epoch 2  | loss: 0.56444 | val_0_auc: 0.69808 |  0:04:25s
epoch 3  | loss: 0.5616  | val_0_auc: 0.69258 |  0:05:52s
epoch 4  | loss: 0.56089 | val_0_auc: 0.69721 |  0:07:19s
epoch 5  | loss: 0.55907 | val_0_auc: 0.69247 |  0:08:47s
epoch 6  | loss: 0.55738 | val_0_auc: 0.6823  |  0:10:14s
epoch 7  | loss: 0.55642 | val_0_auc: 0.69113 |  0:11:41s

Early stopping occurred at epoch 7 with best_epoch = 2 and best_val_0_auc = 0.69808


/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [3, 18, 14, 4, 6, 2, 3, 12, 3] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [4, 4, 4, 4, 4, 4, 4, 4, 4] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [4, 10, 12, 13, 15, 35, 37, 50, 63] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: mask_t

epoch 0  | loss: 0.58171 | val_0_auc: 0.69228 |  0:01:27s
epoch 1  | loss: 0.56984 | val_0_auc: 0.69252 |  0:02:55s
epoch 2  | loss: 0.5668  | val_0_auc: 0.69691 |  0:04:24s
epoch 3  | loss: 0.56389 | val_0_auc: 0.69633 |  0:05:52s
epoch 4  | loss: 0.562   | val_0_auc: 0.69715 |  0:07:18s
epoch 5  | loss: 0.55956 | val_0_auc: 0.69835 |  0:08:45s
epoch 6  | loss: 0.5584  | val_0_auc: 0.69141 |  0:10:11s
epoch 7  | loss: 0.55683 | val_0_auc: 0.69919 |  0:11:38s
epoch 8  | loss: 0.55645 | val_0_auc: 0.68917 |  0:13:04s
epoch 9  | loss: 0.55594 | val_0_auc: 0.70162 |  0:14:31s
epoch 10 | loss: 0.55591 | val_0_auc: 0.70184 |  0:15:58s
epoch 11 | loss: 0.55578 | val_0_auc: 0.69704 |  0:17:24s
epoch 12 | loss: 0.55421 | val_0_auc: 0.69648 |  0:18:51s
epoch 13 | loss: 0.55457 | val_0_auc: 0.6994  |  0:20:17s
epoch 14 | loss: 0.5531  | val_0_auc: 0.69821 |  0:21:43s
Stop training because you reached max_epochs = 15 with best_epoch = 10 and best_val_0_auc = 0.70184


/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [3, 18, 14, 4, 6, 2, 3, 12, 3] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [4, 4, 4, 4, 4, 4, 4, 4, 4] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [4, 10, 12, 13, 15, 35, 37, 50, 63] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: mask_t

...

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_dims changed from [3, 18, 14, 4, 6, 2, 3, 12, 3] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_emb_dim changed from [4, 4, 4, 4, 4, 4, 4, 4, 4] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: cat_idxs changed from [4, 10, 12, 13, 15, 35, 37, 50, 63] to []
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: mask_t

epoch 0  | loss: 0.59648 | val_0_auc: 0.74945 |  0:01:24s
epoch 1  | loss: 0.5841  | val_0_auc: 0.75241 |  0:02:49s
epoch 2  | loss: 0.57909 | val_0_auc: 0.75358 |  0:04:14s
epoch 3  | loss: 0.577   | val_0_auc: 0.75036 |  0:05:38s
epoch 4  | loss: 0.57462 | val_0_auc: 0.75643 |  0:07:04s
epoch 5  | loss: 0.57284 | val_0_auc: 0.75341 |  0:08:28s
epoch 6  | loss: 0.57078 | val_0_auc: 0.74494 |  0:09:52s
epoch 7  | loss: 0.57049 | val_0_auc: 0.74589 |  0:11:18s
epoch 8  | loss: 0.57005 | val_0_auc: 0.74396 |  0:12:45s
epoch 9  | loss: 0.569   | val_0_auc: 0.7489  |  0:14:12s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_val_0_auc = 0.75643


/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


CV AUC scores:  []


ValueError: max() arg is an empty sequence

In [20]:
print("CV AUC scores: ", cv_scores_xgb)
print("Maximum CV AUC score: ", max(cv_scores_xgb))

CV AUC scores:  [0.6980845171251402, 0.7018412794533331, 0.8087590609052192, 0.8092293621649994, 0.7564286305892265]
Maximum CV AUC score:  0.8092293621649994


In [24]:
print(fitted_models_tabnet)

[TabNetClassifier(n_d=8, n_a=8, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='entmax', input_dim=64, output_dim=2, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]), TabNetClassifier(n_d=8, n_a=8, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='entmax', input_dim=64, output_dim=2, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]), TabNetClassifier(n_d=8, n_a=8, n_steps=5, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=

In [27]:
# with open("fitted_models_tabnet.pickle", "wb") as f:
#     pickle.dump(fitted_models_tabnet, f)
    
# for i, m in enumerate(fitted_models_tabnet):
#     saving_path_name = f"/kaggle/working/tabnet_model_test_{i}"
#     saved_filepath = m.save_model(saving_path_name)

Successfully saved model at /kaggle/working/tabnet_model_test_0.zip
Successfully saved model at /kaggle/working/tabnet_model_test_1.zip
Successfully saved model at /kaggle/working/tabnet_model_test_2.zip
Successfully saved model at /kaggle/working/tabnet_model_test_3.zip
Successfully saved model at /kaggle/working/tabnet_model_test_4.zip


In [ ]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:]]
        
        return np.mean(y_preds, axis=0)
    
    def predict_proba_with_dl(self, X1, X2):
        
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:]]
        
        # TODO: ADD DL predict     
        
        
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_cat+fitted_models_lgb+fitted_models_xgb)

# # method1
# catVoting = VotingModel(fitted_models_cat)
# lgbVoting = VotingModel(fitted_models_lgb)
# xgbVoting = VotingModel(fitted_models_xgb)

# RandomizedSearch(catVoting, lgbVoting, xgbVoting, DL1Voting, DL2...)

# # method2
# m = VotingModel(fitted_models_cat+fitted_models_lgb+fitted_models_xgb+fitted_models_dl1+...)

In [ ]:
# dataset = data.copy()
# x_test_d = x_test.copy()
# len_dataset = len(dataset)
# print(len(dataset), len(x_test_d))
# dataset = pd.concat([dataset, x_test_d])

# numeric_features = dataset.select_dtypes(include=np.number).columns
# dataset[numeric_features] = dataset[numeric_features].apply(
#     lambda x: (x-x.mean())/(x.std()))
# dataset[numeric_features] = dataset[numeric_features].fillna(0)

# cat_features = [col for col in dataset.columns if dataset[col].dtype.name == 'category' or dataset[col].dtype.name == 'object']
# dataset = pd.get_dummies(dataset, columns=cat_features, dtype=np.float32)

In [ ]:
# x_test_d = dataset[len_dataset:]
# dataset = dataset[:len_dataset]

In [ ]:
# print(len(x_test_d.columns))
# print(len(dataset.columns))

In [ ]:
# x_train_d, x_valid_d, y_train_d, y_valid_d = train_test_split(df_train, label, test_size=0.3, shuffle=True, stratify=label)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# x_train_d  = torch.tensor(x_train_d.values, dtype=torch.float32).to(device)
# x_valid_d  = torch.tensor(x_valid_d.values, dtype=torch.float32).to(device)
# y_train_d  = torch.tensor(y_train_d.values, dtype=torch.float32).to(device).reshape(-1, 1)
# y_valid_d  = torch.tensor(y_valid_d.values, dtype=torch.float32).reshape(-1, 1)

In [ ]:
# x_test_d = torch.tensor(x_test_d.values, dtype=torch.float32).to(device)

In [ ]:
# del data, dataset
# gc.collect()

# Deep

In [ ]:
# import numpy as np
# import pandas as pd
# import polars as pl
# import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
# from sklearn.metrics import roc_auc_score, accuracy_score

# import torch
# import torch.nn as nn
# from torch.utils import data

# import importlib.util
# import sys

# def load_module_from_path(module_name, file_path):
#     spec = importlib.util.spec_from_file_location(module_name, file_path)
#     module = importlib.util.module_from_spec(spec)
#     sys.modules[module_name] = module
#     spec.loader.exec_module(module)
#     return module

# loaded_module = load_module_from_path('model', '/kaggle/input/ensemble/other/model_fn/1/model.py')

# from model import Transformer, KAN, TransformerKAN
# model_name = "Transformer" # "Transformer", "KAN", "TransformerKAN"
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# PATH = f"/kaggle/working/{model_name}.pth"

In [ ]:
# def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
#     gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
#         .sort_values("WEEK_NUM")\
#         .groupby("WEEK_NUM")[["target", "score"]]\
#         .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
#     x = np.arange(len(gini_in_time))
#     y = gini_in_time
#     a, b = np.polyfit(x, y, 1)
#     y_hat = a*x + b
#     residuals = y - y_hat
#     res_std = np.std(residuals)
#     avg_gini = np.mean(gini_in_time)
#     return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [ ]:
# from tqdm import tqdm

# def load_data(data_array, batch_size, is_train=True):
#     dataset = data.TensorDataset(*data_array)
#     return data.DataLoader(dataset, batch_size, shuffle=is_train)

# def train(model, x_train, y_train, x_valid, y_valid, num_epochs, lr, batch_size, weight_decay):
#     best_validation = 0
#     train_iter = load_data((x_train, y_train), batch_size)
#     val_iter = load_data((x_valid, y_valid), batch_size, is_train=False)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
#     scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
#     # loss = nn.BCELoss()
#     # loss = nn.BCEWithLogitsLoss()
#     loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(5).to(device))
#     # loss = nn.CrossEntropyLoss(weight=torch.tensor(0.7).to(device))
#     for epoch in range(num_epochs):
#         model.train()
#         print("training")
#         for X, y in tqdm(train_iter):
#             # pred = torch.argmax(model(X), dim=1)
#             # pred = model(X)[:,1].reshape(-1, 1)
#             logits = model(X)
#             # print(pred.shape)
#             # print(y)
#             l = loss(logits, y)
#             optimizer.zero_grad()
#             l.backward()
#             optimizer.step()
#         model.eval()
#         y_prob = np.array([])
#         y_pred = np.array([])
#         cur_validation = 0
#         print("validation")
#         with torch.no_grad():
#             for X, y in tqdm(val_iter):
#                 # print(X.shape)
#                 # X = X.view(-1, X.shape[1])
#                 m = model.predict(X)
#                 pred, prob = m[0].flatten(), m[1].flatten()
#                 # pred = m.flatten() 
#                 y_prob = np.concatenate((y_prob, prob.cpu().detach().numpy()))
#                 y_pred = np.concatenate((y_pred, pred.cpu().detach().numpy()))
#                 # print(y_prob.shape)
#                 # print(y_pred.shape)
#             # validation_score.append(validation(y_valid, y_prob))
#             # print(y_pred)
#             cur_validation = validation(y_valid, y_prob)
#             print(f"roc_auc_score: {cur_validation}")
#             print(f"accuracy: {accuracy(y_valid, y_prob)}")
#             if cur_validation > best_validation:
#                 best_validation = cur_validation
#                 print("Save new model")
#                 torch.save(model, PATH)
#         # scheduler.step()
    
#     return y_prob, y_pred

# def validation(y_valid, y_score):
#     return roc_auc_score(y_true=y_valid, y_score=y_score)

# # only use to get test batch
# def get_batch(x, batch_size):
#     ix = torch.arange(0, len(x), batch_size)
#     return [x[i:i+batch_size] for i in ix]

# def accuracy(y_true, y_pred):
#     y_pred = np.where(y_pred > 0.5, 1., 0.)
#     # y_pred = np.argmax(y_pred, dim=-1)
#     # y_pred[y_pred > 0.5] = 1.
#     # y_pred[y_pred <= 0.5] = 0
#     # print(np.sum(y_pred))
#     return accuracy_score(y_true=y_true, y_pred=y_pred)

# def predict(model, x_test, batch_size):
#     test_iter = get_batch(x_test, batch_size)
#     res = []
#     model.eval()
#     with torch.no_grad():
#         for X in test_iter:
#             X = X.to(device)  # 確保數據位於相同設備
#             label, prob = model.predict(X)
# #             print(prob.shape)
#             res.append(prob.flatten().cpu().numpy())  # 將結果移回 CPU 並存入 res
#     return np.concatenate(res)[:len(x_test)]  # 確保輸出與輸入長度相同

In [ ]:
# num_epoch, lr, batch_size, weight_decay = 5, 1e-3, 64, 1e-4

# # "Transformer", "KAN", "TransformerKAN"
# if model_name == "Transformer":
#     print("Train Transformer")
#     model = Transformer(in_features=len(x_train_d[0]), drop=0.).to(device)
# elif model_name == "KAN":
#     print("Train KAN")
#     model = KAN([len(x_train_d[0]), batch_size, 1]).to(device)
# elif model_name == "TransformerKAN":
#     print("Train TransformerKAN")
#     model = TransformerKAN(in_features=len(x_train_d[0]), drop=0.).to(device)


# prob, pred = train(model, x_train_d, y_train_d, x_valid_d, y_valid_d, num_epoch, lr, batch_size, weight_decay)
# # print(score)

In [ ]:
# gc.collect()

# Load Model

In [ ]:
# # with open(f'/kaggle/models/lgbm_gbdt_model_07_train_0.7837037039189849.pickle', 'rb') as f:
# #     cls = pickle.load(f)

# # with open(f'/kaggle/models/xgboost_model_07_train_0.7638036661339924.pickle', 'rb') as f:
# #     xgb_model = pickle.load(f)

# # with open(f'/kaggle/models/catboost_model_07_train_0.8088802180545341.pickle', 'rb') as f:
# #     cat_model = pickle.load(f)

# # with open(f'/kaggle/input/ensemble/scikitlearn/treebase/1/lgbm_gbdt_model_07_train_0.7837037039189849.pickle', 'rb') as f:
# #     cls = pickle.load(f)

# # with open(f'/kaggle/input/ensemble/scikitlearn/treebase/1/xgboost_model_07_train_0.7638036661339924.pickle', 'rb') as f:
# #     xgb_model = pickle.load(f)

# # with open(f'/kaggle/input/ensemble/scikitlearn/treebase/1/catboost_model_07_train_0.8088802180545341.pickle', 'rb') as f:
# #     cat_model = pickle.load(f)

# with open(f'/kaggle/input/ensemble/scikitlearn/random_features/2/lgbm_gbdt_model_07_train_0.7624031332281023.pickle', 'rb') as f:
#     cls = pickle.load(f)

# with open(f'/kaggle/input/ensemble/scikitlearn/random_features/2/xgboost_model_07_train_0.7374121828487601.pickle', 'rb') as f:
#     xgb_model = pickle.load(f)

# with open(f'/kaggle/input/ensemble/scikitlearn/random_features/2/catboost_model_07_train_0.7767447884537366.pickle', 'rb') as f:
#     cat_model = pickle.load(f)


# transformer_model = torch.load('/kaggle/input/ensemble/pytorch/dl/4/Transformer.pth')    
# kan_model = torch.load('/kaggle/input/ensemble/pytorch/dl/4/KAN.pth')
# transformerKan_model = torch.load('/kaggle/input/ensemble/pytorch/dl/4/TransformerKAN.pth')   


In [ ]:
# pred = cls.predict(x_valid)
# auc = roc_auc_score(y_true=y_valid, y_score=pred)
# print(auc)

# xgb_pred = xgb_model.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_true=y_valid, y_score=xgb_pred)
# print(auc)

# cat_pred = cat_model.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_true=y_valid, y_score=cat_pred)
# print(auc)


In [ ]:
# gc.collect()

In [ ]:
# transformer_pred = predict(transformer_model, x_valid_d, 64)
# auc = roc_auc_score(y_true=y_valid_d, y_score=transformer_pred)
# print(auc)

In [ ]:
# kan_pred = predict(kan_model, x_valid_d, 64)
# auc = roc_auc_score(y_true=y_valid_d, y_score=kan_pred)
# print(auc)

In [ ]:
# transformerKan_pred = predict(transformerKan_model, x_valid_d, 64)
# auc = roc_auc_score(y_true=y_valid_d, y_score=transformerKan_pred)
# print(auc)

# Ensemble (3 tree + 3 dl)

In [ ]:
# # use randomized search instead of linear search to save time
# def RandomizedSearch(n_init, pred1, pred2, pred3, pred4, pred5, pred6, y_true, random_state=None):
#     if random_state:
#         np.random.seed(random_state)
    
#     weight1 = np.arange(0, 20, 1)
#     weight2 = np.arange(0, 20, 1)
#     weight3 = np.arange(0, 20, 1)
#     weight4 = np.arange(0, 20, 1)
#     weight5 = np.arange(0, 20, 1)
#     weight6 = np.arange(0, 20, 1)

#     df = pd.DataFrame(columns=['weight1', 'weight2', 'weight3', 'weight4', 'weight5', 'weight6', 'score'])
#     for i in range(n_init):
#         # pick weight
#         w1 = np.random.choice(weight1, replace=True) 
#         w2 = np.random.choice(weight2, replace=True) 
#         w3 = np.random.choice(weight3, replace=True) 
#         w4 = np.random.choice(weight4, replace=True) 
#         w5 = np.random.choice(weight4, replace=True) 
#         w6 = np.random.choice(weight4, replace=True) 

#         y_ensemble = (w1*pred1 + w2*pred2 + w3*pred3 + w4*pred4 + w5*pred5 + w6*pred6)/(w1+w2+w3+w4+w5+w6)
#         score = roc_auc_score(y_true=y_true, y_score=y_ensemble)
        
#         df.loc[i] = [w1, w2, w3, w4, w5, w6, score]
        
#     return df


In [ ]:
# # use randomized search instead of linear search to save time
# def RandomizedSearch(n_init, pred1, pred2, pred3, pred4, y_true, random_state=None):
#     if random_state:
#         np.random.seed(random_state)
    
#     weight1 = np.arange(0, 20, 1)
#     weight2 = np.arange(0, 20, 1)
#     weight3 = np.arange(0, 20, 1)
#     weight4 = np.arange(0, 20, 1)

#     df = pd.DataFrame(columns=['weight1', 'weight2', 'weight3', 'weight4', 'score'])
#     for i in range(n_init):
#         # pick weight
#         w1 = np.random.choice(weight1, replace=True) 
#         w2 = np.random.choice(weight2, replace=True) 
#         w3 = np.random.choice(weight3, replace=True) 
#         w4 = np.random.choice(weight4, replace=True) 

        
#         y_ensemble = (w1*pred1 + w2*pred2 + w3*pred3 + w4*pred)/(w1+w2+w3+w4)
#         score = roc_auc_score(y_true=y_true, y_score=y_ensemble)
        
#         df.loc[i] = [w1, w2, w3, w4, score]
        
#     return df

In [ ]:
# # use randomized search instead of linear search to save time
# def RandomizedSearch(n_init, pred1, pred2, pred3, y_true, random_state=None):
#     if random_state:
#         np.random.seed(random_state)
    
#     weight1 = np.arange(0, 20, 1)
#     weight2 = np.arange(0, 20, 1)
#     weight3 = np.arange(0, 20, 1)

#     df = pd.DataFrame(columns=['weight1', 'weight2', 'weight3', 'score'])
#     for i in range(n_init):
#         # pick weight
#         w1 = np.random.choice(weight1, replace=True) 
#         w2 = np.random.choice(weight2, replace=True) 
#         w3 = np.random.choice(weight3, replace=True) 

        
#         y_ensemble = (w1*pred1 + w2*pred2 + w3*pred3)/(w1+w2+w3+w4)
#         score = roc_auc_score(y_true=y_true, y_score=y_ensemble)
        
#         df.loc[i] = [w1, w2, w3, score]
        
#     return df

In [ ]:
# df = RandomizedSearch(n_init=100, pred1=pred, pred2=xgb_pred, pred3=cat_pred, pred4=transformer_pred, pred5=kan_pred, pred6=transformerKan_pred, y_true=y_valid, random_state=8787)
# best_weights = df.sort_values(by=['score'], ascending=False).iloc[0]
# df.sort_values(by=['score'], ascending=False)

# Test data predict (Baseline)

In [ ]:
# y_test = cls.predict(x_test)
# y_test

In [ ]:
# test_ensemble = (best_weights['weight1'] * cls.predict(x_test) + best_weights['weight2'] * xgb_model.predict_proba(x_test)[:,1] + best_weights['weight3'] * cat_model.predict_proba(x_test)[:,1] + best_weights['weight4'] * predict(transformer_model, x_test_d, 64) + best_weights['weight5'] * predict(kan_model, x_test_d, 64) + best_weights['weight6'] * predict(transformerKan_model, x_test_d, 64)) / (best_weights['weight1']+best_weights['weight2']+best_weights['weight3']+best_weights['weight4']+best_weights['weight5']+best_weights['weight6'])
# test_ensemble

In [ ]:
# now = datetime.now()
# now = now.strftime("%Y-%m-%dT%H_%M_%S")

In [ ]:
# test_df = pd.DataFrame(columns=['case_id', 'score'])
# test_df['case_id'] = x_test.index
# test_df['score'] = test_ensemble

# # test_df.to_csv(f'/kaggle/submission/submission_{now}.csv', index=False)
# test_df.to_csv('submission.csv', index=False)
# test_df